In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



In [26]:
# Load the image
image_path = 'dataset/minions.jpg'
img = cv2.imread(image_path)

if img is None:
    raise FileNotFoundError(f"Error: Unable to load image at {image_path}. Please check the path.")

In [29]:
# Convert the original image to RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


# Convert the original image to grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Specify the parameters for our HOG descriptor
win_size = img_gray.shape
cell_size = (8, 8)
block_size = (16, 16)
block_stride = (8, 8)
num_bins = 9

# Set the parameters of the HOG descriptor using the variables defined above
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

# Compute the HOG Descriptor for the gray scale image
hog_descriptor = hog.compute(img_gray)

print ('HOG Descriptor:', hog_descriptor)
print ('HOG Descriptor has shape:', hog_descriptor.shape)


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\hog.cpp:95: error: (-215:Assertion failed) (winSize.width - blockSize.width) % blockStride.width == 0 && (winSize.height - blockSize.height) % blockStride.height == 0 in function 'cv::HOGDescriptor::getDescriptorSize'
